# Test Script for QOSST Integration

This script simulates IR for Gaussian modulated / 4-state modulated CV-QKD system as described in [1] for fixed reconciliation efficiency $\beta$.

In [6]:
import sys
sys.path.append('../build')

Import the modules to be used:

In [5]:
import information_reconciliation as ir
import numpy as np

#AWGN channel capacity
def capacity(SNR): 
    return 0.5 * np.log2(1 + (10**(SNR/10)))

Define the simulation variables

In [7]:
N = 2e7 # Number of quantum states to be sent
d = 8 # MDR dimension (can be set in the parameter file)
rate = 0.2 # code rate of the error correction code. The library uses the reconciliation efficiency, but for the sake of this example we will use the code rate
SNR_min = -4.75
SNR_max = -4.45
total_error_count = 256 # Total number of frame errors

Run the simulation

In [8]:

for SNR in np.arange(SNR_min, SNR_max, 0.05):
    simulated_error_count = 0
    beta = rate/capacity(SNR) # the library uses the reconciliation efficiency instead of the rate
    sigma_square = 10**(-SNR/10) # needed for quantum channel simulation
    simulated_frames = 0
    
    while simulated_error_count<total_error_count:
        #sample the transmitted states as Gaussian
        X = np.random.normal(0, 1, int(N))
        
        #sample the transmitted states as 4-state (BPSK modulated)
        #X = np.random.choice([1, -1], int(N)) 
        
        #Quantum channel
        Y = X + np.sqrt(sigma_square)*np.random.normal(0, 1, int(N))

        #Step 1: Bob does MDR
        [channel_message,syndrome, normalization_vector, raw_key] = ir.reconcile_Bob(bob_states=Y, beta=beta, SNR=SNR, MDR_dim=d)
        
        #Step 2: Bob sends the classical channel message, syndrome and the normalization vector to Alice
        
        #Step 3: Alice does MDR with the classical channel message, syndrome, and normalization vector. CRC_Alice is the CRC of the decoded frames. Discard_flag is a boolean array that indicates whether the corresponding frame could be decoded. Decoded frames are the frames that the decoder converged. We assume that the signal to noise ratio is known to Alice and Bob.
        [CRC_Alice, discard_flag, decoded_frames] = ir.reconcile_Alice(alice_states=X, classical_channel_message=channel_message, syndrome=syndrome, normalization_vector=normalization_vector, SNR=SNR, MDR_dim=d)
        
        num_of_frames=len(discard_flag)
        
        #Step 4: Alice sends the CRC_Alice and discard_flag to Bob
        
        #Step 5: Bob compares the CRCs of the decoded frames and the raw keys. If they match, the frames are kept, otherwise they are discarded. 
        [final_discard_flag, Bob_final_keys] = ir.CRC_check_Bob(raw_keys=raw_key, CRC_Alice=CRC_Alice, discard_flag=discard_flag)
        
        #Step 6: Bob sends the final_discard_flag to Alice. It includes both the frames that could not be decoded by Alice and the frames that were discarded in the CRC check.
        
        #Step 7: Alice discards the frames that Bob discarded
        Alice_final_keys = [frame for frame, flag in zip(decoded_frames, final_discard_flag) if flag == 0]
        
        #Safety check 1: Compare the keys of the Alice and Bob and make sure that they are the same
        Alice_final_keys_np = np.array(Alice_final_keys)
        Bob_final_keys_np = np.array(Bob_final_keys)
        
        if np.array_equal(Alice_final_keys_np, Bob_final_keys_np):
            pass
        else:
            print("Error")
            break
        
        #Safety Check 2: Count the number of errors to compare it with the Fig.2 in [1]
        simulated_error_count += sum(final_discard_flag)
        simulated_frames += num_of_frames
                
    print("SNR: ", "{:.2f}".format(SNR), "dB  FER: ", "{:.5f}".format((simulated_error_count/simulated_frames)), "Simulated error count: ", simulated_error_count, "Simulated frames: ", simulated_frames)

SNR:  -4.75 dB  FER:  0.98750 Simulated error count:  395 Simulated frames:  400
SNR:  -4.70 dB  FER:  0.92000 Simulated error count:  368 Simulated frames:  400
SNR:  -4.65 dB  FER:  0.78750 Simulated error count:  315 Simulated frames:  400
SNR:  -4.60 dB  FER:  0.47167 Simulated error count:  283 Simulated frames:  600
SNR:  -4.55 dB  FER:  0.19429 Simulated error count:  272 Simulated frames:  1400
SNR:  -4.50 dB  FER:  0.06000 Simulated error count:  264 Simulated frames:  4400


[1] E. E. Cil and L. Schmalen, _"An open-source library for information reconciliation in continuous-variable QKD,"_ Proc. International Conference on Quantum Cryptography (QCRYPT), Vigo, Spain, Sep. 2024, poster presentation